# Document search with embeddings

## Overview

This example demonstrates how to use the Gemini API to create embeddings so that you can perform document search. You will use the Python client library to build a word embedding that allows you to compare query strings or questions, to document contents.

In this Notebook, we will use embeddings to perform document search over a set of documents to ask questions related to the Google Car. **This Notebook in way demonstrates the concept of Semantic Search as used in Vector Databases.**


## Setup

First, download and install the Gemini API Python library.

In [1]:
!pip install -U -q google.generativeai

In [ ]:
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm
## The purpose of import google.ai.generativelanguage as glm is to import
## the Google AI Generative Language API client library.
## This library allows you to programmatically interact with the Generative Language API,
## which provides access to state-of-the-art text generation and manipulation models.

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown

### Create an API Key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [ ]:
# Or use `os.getenv('API_KEY')` to fetch an environment variable.
from google.colab import userdata
#userdata.get('API_KEY')
API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=API_KEY)

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.

**Note**: At this time, the Gemini API is [only available in certain regions](https://developers.generativeai.google/available_regions).

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

## models/embedding-001 is a pre-trained embedding model for natural language processing tasks.
## It is designed to convert words and phrases into numerical vectors, allowing machines
## to better understand the context and meaning of text.

## models/embedding-001 can be used for a wide range of NLP tasks, including:
## * Text classification
## * Named entity recognition
## * Natural language inference
## * Question answering
## * Machine translation

models/embedding-001


## Embedding generation

In this section, you will see how to generate embeddings for a piece of text using the embeddings from the Gemini API.

For the embeddings model, embedding-001, there is a new task type parameter and the optional title (only valid with task_type=`RETRIEVAL_DOCUMENT`).

These new parameters apply only to the newest embeddings models.The task types are:

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

Note: Specifying a `title` for `RETRIEVAL_DOCUMENT` provides better quality embeddings for retrieval.

In [ ]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

model = 'models/embedding-001'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)

print("size of the vector =",  len(embedding['embedding']))

## Note
## RETRIEVAL_QUERY: This attribute is assigned to text that represents a user's search query.
## It helps NLP models understand that the text is a query and should be treated differently
## from other types of text, such as documents.

## RETRIEVAL_DOCUMENT: This attribute is assigned to text that represents a document
## or piece of content that is being retrieved or searched.
## It helps NLP models understand that the text is a document
## and should be analyzed for its relevance to the query.


## Explanation of what the code does:
## It loads a pre-trained embedding model called models/embedding-001. This model is trained to convert text into numerical vectors.
## It sets the task_type to "retrieval_document". This tells the model that the input text is a document in a search or retrieval setting.
## It generates an embedding for the sample_text. This embedding is a numerical vector that captures the semantic meaning and context of the text.


{'embedding': [0.03411343, -0.05517662, -0.020209055, -0.0041249567, 0.058917783, 0.014129515, 0.0045353593, 0.0014303668, 0.05976634, 0.08292115, 0.007162964, 0.0069041685, -0.053083427, -0.010905125, 0.0321402, -0.037163995, 0.050372455, 0.019348344, -0.037328612, 0.026647927, 0.030781753, -0.011288501, -0.031485256, -0.060248993, -0.026219442, -0.009794235, 0.006630139, -0.01846516, -0.026324715, 0.020442624, -0.06317684, 0.014559574, -0.052296035, 0.016451128, -9.720217e-05, -0.051706687, -0.0054406044, -0.056967627, 0.011144145, -0.009201792, -0.0021951047, -0.1099701, -0.011712193, 0.021221714, 0.009171804, -0.029621972, 0.034534883, 0.039578073, 0.019021519, -0.06269169, 0.039473332, 0.052403256, 0.061814185, -0.034507945, -0.009557816, -0.0049551064, 0.017839009, -0.021176832, 0.015043588, 0.015390569, -0.006334281, 0.043696404, -0.028341983, 0.028433999, 0.01472686, -0.06585564, -0.044533554, 0.0055523133, 0.035775978, 0.031099156, 0.027357662, 0.028062241, 0.056972917, -0.054

## Building an embeddings database

Here are three sample texts to use to build the embeddings database. You will use the Gemini API to create embeddings of each of the documents. Turn them into a dataframe for better visualization.

In [ ]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Google car has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Google car has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Google car has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]
#documents = [DOCUMENT1]

Organize the contents of the dictionary into a dataframe for better visualization.

In [ ]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Operating the Climate Control System,Your Google car has a climate control system t...
1,Touchscreen,Your Google car has a large touchscreen displa...
2,Shifting Gears,Your Google car has an automatic transmission....


Get the embeddings for each of these bodies of text. Add this information to the dataframe.

In [ ]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,Operating the Climate Control System,Your Google car has a climate control system t...,"[-0.038105167, -0.027855955, -0.044773404, -0...."
1,Touchscreen,Your Google car has a large touchscreen displa...,"[0.0019631658, -0.03915825, -0.014636642, -0.0..."
2,Shifting Gears,Your Google car has an automatic transmission....,"[-0.048327737, -0.011152455, -0.028626956, -0...."


## Document search with Q&A

Now that the embeddings are generated, let's create a Q&A system to search these documents. You will ask a question, create an embedding of the question, and compare it against the collection of embeddings in the dataframe.

The embedding of the question will be a vector (list of float values), which will be compared against the vector of the documents using the dot product. This vector returned from the API is already normalized. The dot product represents the similarity in direction between two vectors.

The values of the dot product can range between -1 and 1, inclusive. If the dot product between two vectors is 1, then the vectors are in the same direction. If the dot product value is 0, then these vectors are orthogonal, or unrelated, to each other. Lastly, if the dot product is -1, then the vectors point in the opposite direction and are not similar to each other.

Note, with the new embeddings model (`embedding-001`), specify the task type as `QUERY` for user query and `DOCUMENT` when embedding a document text.

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.

In [ ]:
query = "How do you shift gears in the Google car?"
model = 'models/embedding-001'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

Use the `find_best_passage` function to calculate the dot products, and then sort the dataframe from the largest to smallest dot product value to retrieve the relevant passage out of the database.

In [ ]:
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

View the most relevant document from the database:

In [ ]:
passage = find_best_passage(query, df)
passage

'Your Google car has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.'

## Question and Answering Application

Let's define a make_prompt function to be used in our **Q & A** system. Input your own custom data below to create a simple question and answering example. You will still use the dot product as a metric of similarity.

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the question, then you print - Sorry cannot answer.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

 ## The textwrap.dedent() function in Python is used to remove common leading whitespace
 ## from every line in a multiline string. It is useful for removing indentation
 ## from a string that has been indented relative to the left margin,
 ## making it easier to read and process the string.

In [ ]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the question, then you print - Sorry cannot answer.
  QUESTION: 'How do you shift gears in the Google car?'
  PASSAGE: 'Your Google car has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This posi

Choose one of the Gemini content generation models in order to find the answer to your query.

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')
answer = model.generate_content(prompt)

In [ ]:
Markdown(answer.text)

- Sorry cannot answer. The provided text does not contain any information about Google cars.

## Practical Use Case

**Building a Q&A System for Banking and Finance**

You want to create a simple Q&A system specifically focused on the banking and finance sector. You will utilize your existing dataset within this domain and leverage the capabilities of a pre-trained model like Gemini-Pro.

**Your task involves the following steps:**

1. **Data Preprocessing:** Clean and prepare your banking and finance dataset for the chosen model. This may involve tasks like handling missing values, normalization, or tokenization.
2. **Fine-tuning Gemini-Pro:** Utilize the pre-trained Gemini-Pro model and fine-tune it on your prepared dataset. This process aims to adapt the model's knowledge to the specific domain of banking and finance.
3. **Embedding Generation:** Generate embeddings for both your questions and documents in your dataset. Embeddings represent the meaning of text in a numerical vector format, which is crucial for similarity comparisons.
4. **Q&A System Design:** Design and implement the core functionalities of your Q&A system. This includes accepting user questions, generating embeddings for the questions, comparing them with document embeddings, and retrieving the most relevant documents or snippets as answers.
5. **Evaluation:** Evaluate the performance of your Q&A system. Choose appropriate metrics to assess the effectiveness of your system in answering user queries accurately and efficiently.

**Bonus Challenge:**

* Explore techniques to improve your system's robustness, such as handling ambiguous questions or incorporating context in the search process.
* Investigate the use of interactive elements or visualizations to enhance the user experience of your Q&A system.

**Remember to document your work clearly at each stage, including code snippets and results. This will help you track your progress and evaluate the effectiveness of your approach.**

This question encourages the learner to apply their knowledge of data preparation, fine-tuning, and embedding generation in a practical context. It also challenges them to design, implement, and evaluate a real-world application.

In [ ]:
DOCUMENT1 = {
    "title": "What is a savings account?",
    "content": "A savings account is a type of bank account that allows you to deposit and withdraw money. Savings accounts typically offer a higher interest rate than checking accounts, but they may have restrictions on how often you can withdraw money. Savings accounts are a good option for people who want to save money for a future goal, such as a down payment on a house or a new car."
}

DOCUMENT2 = {
    "title": "What is a checking account?",
    "content": "A checking account is a type of bank account that allows you to deposit and withdraw money, as well as write checks. Checking accounts typically offer a lower interest rate than savings accounts, but they may have fewer restrictions on how often you can withdraw money. Checking accounts are a good option for people who need to access their money frequently."
}

DOCUMENT3 = {
    "title": "What is a fixed deposit?",
    "content": "A fixed deposit is a type of investment that allows you to deposit money for a fixed period of time. Fixed deposits typically offer a higher interest rate than savings accounts, but you cannot access your money until the end of the term. Fixed deposits are a good option for people who want to save money for a specific goal, such as a child's education or a retirement fund."
}

DOCUMENT4 = {
    "title": "What is a loan?",
    "content": "A loan is a type of financial assistance that allows you to borrow money from a bank or other lender. Loans typically have a fixed interest rate and a fixed repayment period. Loans can be used for a variety of purposes, such as buying a house, a car, or starting a business. Loans are a good option for people who need to borrow money for a specific purpose."
}

DOCUMENT5 = {
    "title": "What is a credit card?",
    "content": "A credit card is a type of payment card that allows you to borrow money from a bank or other lender. Credit cards typically have a high interest rate, but they offer a number of benefits, such as rewards points and cash back. Credit cards are a good option for people who need to make purchases without having to pay for them immediately."
}

DOCUMENT6 = {
    "title": "What is a debit card?",
    "content": "A debit card is a type of payment card that allows you to access your money directly from your bank account. Debit cards typically have a low interest rate, but they may have some restrictions on how often you can use them. Debit cards are a good option for people who want to avoid paying interest on their purchases."
}

DOCUMENT7 = {
    "title": "What is a mutual fund?",
    "content": "A mutual fund is a type of investment that allows you to pool your money with other investors to invest in a variety of stocks, bonds, and other assets. Mutual funds are a good option for people who want to diversify their investments and reduce their risk."
}

DOCUMENT8 = {
    "title": "What is an exchange-traded fund (ETF)?",
    "content": "An exchange-traded fund (ETF) is a type of investment that tracks a basket of stocks, bonds, or other assets. ETFs are traded on stock exchanges, like stocks, and they offer a number of benefits, such as low costs and diversification. ETFs are a good option for people who want to invest in a specific market or sector."
}

DOCUMENT9 = {
    "title": "What is a pension plan?",
    "content": "A pension plan is a type of retirement savings plan that allows you to save money for your retirement. Pension plans typically offer a number of benefits, such as tax breaks and guaranteed income in retirement. Pension plans are a good option for people who want to save for their retirement and reduce their risk."
}

DOCUMENT10 = {
    "title": "What is a life insurance policy?",
    "content": "A life insurance policy is a type of insurance that provides financial protection to your loved ones in the event of your death. Life insurance policies typically offer a variety of benefits, such as death benefits, disability benefits, and long-term care benefits. Life insurance policies are a good option for people who want to protect their loved ones from financial hardship in the event of their death."
}

#documentsBFSI = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4, DOCUMENT5, DOCUMENT6, DOCUMENT7, DOCUMENT8, DOCUMENT9, DOCUMENT10]


documentsBFSI = []

# Assuming DOCUMENT variables follow a specific naming convention (e.g., DOCUMENT + str(i))
for i in range(1, 11):
  documentsBFSI.append(eval("DOCUMENT" + str(i)))  # Using eval to dynamically access DOCUMENT variables

# Create a dictionary with the extracted data
data = {
    "Title": [doc["title"] for doc in documentsBFSI],
    "Text": [doc["content"] for doc in documentsBFSI]
}

# Create the DataFrame
df1 = pd.DataFrame(data)

df1.head()


,Title,Text
0,What is a savings account?,A savings account is a type of bank account th...
1,What is a checking account?,A checking account is a type of bank account t...
2,What is a fixed deposit?,A fixed deposit is a type of investment that a...
3,What is a loan?,A loan is a type of financial assistance that ...
4,What is a credit card?,A credit card is a type of payment card that a...


In [ ]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model = 'models/embedding-001',
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df1['Embeddings'] = df1.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df1.head()

,Title,Text,Embeddings
0,What is a savings account?,A savings account is a type of bank account th...,"[0.008011587, -0.047471076, -0.06416773, -0.03..."
1,What is a checking account?,A checking account is a type of bank account t...,"[0.019765917, -0.05731653, -0.07145462, -0.028..."
2,What is a fixed deposit?,A fixed deposit is a type of investment that a...,"[0.019940985, -0.037196085, -0.051888086, 0.00..."
3,What is a loan?,A loan is a type of financial assistance that ...,"[0.0007942434, -0.03684029, -0.06654184, 0.006..."
4,What is a credit card?,A credit card is a type of payment card that a...,"[-0.0022393346, -0.084505826, -0.06505255, -0...."


In [ ]:
querybfsi1 = "For how much should I buy an Insurance policy?"
model = 'models/embedding-001'

requestbfsi = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

In [ ]:
passagebfsi_1 = find_best_passage(querybfsi1, df1)
passagebfsi_1

'A life insurance policy is a type of insurance that provides financial protection to your loved ones in the event of your death. Life insurance policies typically offer a variety of benefits, such as death benefits, disability benefits, and long-term care benefits. Life insurance policies are a good option for people who want to protect their loved ones from financial hardship in the event of their death.'

In [ ]:
promptbfsi_1 = make_prompt(querybfsi1, passagebfsi_1)
print(promptbfsi_1)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the question, then you print - Sorry cannot answer.
  QUESTION: 'For how much should I buy an Insurance policy?'
  PASSAGE: 'A life insurance policy is a type of insurance that provides financial protection to your loved ones in the event of your death. Life insurance policies typically offer a variety of benefits, such as death benefits, disability benefits, and long-term care benefits. Life insurance policies are a good option for people who want to protect their loved ones from financial hardship in the event of their death.'

    ANSWER:



In [ ]:
model = genai.GenerativeModel('gemini-pro')
answerbfsi = model.generate_content(promptbfsi_1)

Markdown(answerbfsi.text)

ERROR:tornado.access:500 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3578.34ms


InternalServerError: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [ ]:
## Let us change the question , so that LLM can find an answer from our given training content.

querybfsi2 = "Explain me about Life Insurance policy?"
model = 'models/embedding-001'

requestbfsi = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

passagebfsi_2 = find_best_passage(querybfsi2, df1)


promptbfsi_2 = make_prompt(querybfsi2, passagebfsi_2)
print(promptbfsi_2)

model = genai.GenerativeModel('gemini-pro')
answerbfsi_2 = model.generate_content(promptbfsi_2)

Markdown(answerbfsi_2.text)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the question, then you print - Sorry cannot answer.
  QUESTION: 'Explain me about Life Insurance policy?'
  PASSAGE: 'A life insurance policy is a type of insurance that provides financial protection to your loved ones in the event of your death. Life insurance policies typically offer a variety of benefits, such as death benefits, disability benefits, and long-term care benefits. Life insurance policies are a good option for people who want to protect their loved ones from financial hardship in the event of their death.'

    ANSWER:



Life insurance is a smart choice for people who want to ensure their loved ones are not left with financial hardship in the event of their death. There are a variety of benefits to having a life insurance policy, including death benefits, disability benefits, and long-term care benefits.

## Bonus Learning

### Vector Databases for LLMs

**Vector databases** are specialized databases designed to store and efficiently retrieve high-dimensional data represented as vectors. These vectors are arrays of numbers that encode information, making them ideal for representing the complex relationships and semantic meaning captured by LLMs.

### Why Vector Databases are Essential for LLMs

Traditional relational databases, which store data in tables with rows and columns, struggle with the following challenges when dealing with LLM data:

> **High Dimensionality**: LLM outputs and inputs can have thousands or even millions of dimensions, making them cumbersome and inefficient to store in relational databases.

> **Similarity Search**: LLMs often require finding similar data points based on meaning rather than exact matches.
Relational databases lack efficient methods for semantic search.

### Benefits of Vector Databases for LLMs:

Vector databases offer several advantages for LLMs:

> **Efficient Storage and Retrieval**: Vectors are compact representations of information, making them more space-efficient than storing raw text or code in relational databases. Vector databases are also optimized for similarity searches based on vector distances, enabling efficient retrieval of similar LLM outputs or prompts.

> **Semantic Search**: Unlike relational databases that search based on exact matches, vector databases can find data points with similar meaning even if the wording differs. This is crucial for LLMs, which often deal with nuanced language and require understanding context and intent.

> **Scalability**: Vector databases are designed to scale effectively with large datasets of LLM outputs and prompts, making them suitable for complex real-world applications.

### Must Visit Links for Quickly Learning about Vector DB

https://medium.com/data-and-beyond/vector-databases-a-beginners-guide-b050cbbe9ca0


### Additional Resource link for Vector DB
Vector databases use special search techniques known as Approximate Nearest Neighbor (ANN) search, which includes methods like hashing and graph-based searches.

Read more here :
https://www.datacamp.com/blog/the-top-5-vector-databases

